<a href="https://colab.research.google.com/github/AndyZhang127/temp/blob/main/Chipotle_Solution_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 61.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a3134ad674d0a01b52616fd78f28d77958927d19943acdde6c4f0279e5e8c028
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
#step1.Import the necessary libraries

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark=SparkSession.builder.appName('ex2').getOrCreate()
spark

In [4]:
#Step2.import the dataaset from this address
url='https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
spark.sparkContext.addFile(url)

In [5]:
#Step3.Assign it to a variable called chipo
chio=spark.read.csv(SparkFiles.get("chipotle.tsv"),header=True,sep='\t',inferSchema=True)

In [6]:
#Step4.See the first 10 entires
chio.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

In [7]:
chio.head(2)

[Row(order_id=1, quantity=1, item_name='Chips and Fresh Tomato Salsa', choice_description='NULL', item_price='$2.39 '),
 Row(order_id=1, quantity=1, item_name='Izze', choice_description='[Clementine]', item_price='$3.39 ')]

In [8]:
chio.take(2)

[Row(order_id=1, quantity=1, item_name='Chips and Fresh Tomato Salsa', choice_description='NULL', item_price='$2.39 '),
 Row(order_id=1, quantity=1, item_name='Izze', choice_description='[Clementine]', item_price='$3.39 ')]

In [9]:
#Step 5. What is the number of observations in the dataset?
#Solution 1
print(chio.count(),',',len(chio.columns))

4622 , 5


In [10]:
#Solution 2
def sparkShape(dataFrame):
  return(dataFrame.count(),len(dataFrame.columns))

pyspark.sql.dataframe.DataFrame.shape=sparkShape

print(chio.shape())

(4622, 5)


In [11]:
#Step 6. What is the number of columns in the dataset?
len(chio.columns)

5

In [12]:
#Step 7. Print the name of all the columns
chio.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

In [ ]:
#Step 8. How is the dataset indexed?


In [13]:
#Step 9. Which was the most-ordered item?
most_ordered_item=chio.groupBy("item_name").sum()
most_ordered_item.orderBy("sum(quantity)",ascending=0).show(1)

+------------+-------------+-------------+
|   item_name|sum(order_id)|sum(quantity)|
+------------+-------------+-------------+
|Chicken Bowl|       713926|          761|
+------------+-------------+-------------+
only showing top 1 row



In [14]:
#Step 10.For the most-ordered item, how many items were ordered?
most_ordered_item=chio.groupBy("item_name").sum()
most_ordered_item.orderBy("sum(quantity)",ascending=0).show(1)

+------------+-------------+-------------+
|   item_name|sum(order_id)|sum(quantity)|
+------------+-------------+-------------+
|Chicken Bowl|       713926|          761|
+------------+-------------+-------------+
only showing top 1 row



In [15]:
#Step 11. What was the most ordered item in the choice_description column?
most_ordered_item_de=chio.groupBy("choice_description").sum()
most_ordered_item_de.orderBy("sum(quantity)",ascending=0).show(2)

+------------------+-------------+-------------+
|choice_description|sum(order_id)|sum(quantity)|
+------------------+-------------+-------------+
|              NULL|      1178731|         1382|
|       [Diet Coke]|       123455|          159|
+------------------+-------------+-------------+
only showing top 2 rows



In [16]:
#Step 12. How many items were orderd in total?
total_quantity = chio.agg({"quantity":"sum"})
total_quantity.show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



In [17]:
#Step 13.Turn the item price into a float
#Step 13-a. Check the item price type
from pyspark.sql.functions import regexp_replace

chio.show(2)
chio.printSchema()

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 |
|       1|       1|                Izze|      [Clementine]|    $3.39 |
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



In [18]:
#Step 13.b. Create a lambda function and change the type of item price

#solution1
from pyspark.sql.functions import regexp_replace

price=chio.withColumn("price",regexp_replace("item_price","\$"," "))
price.show(2)
price.printSchema()

+--------+--------+--------------------+------------------+----------+------+
|order_id|quantity|           item_name|choice_description|item_price| price|
+--------+--------+--------------------+------------------+----------+------+
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 | 2.39 |
|       1|       1|                Izze|      [Clementine]|    $3.39 | 3.39 |
+--------+--------+--------------------+------------------+----------+------+
only showing top 2 rows

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)
 |-- price: string (nullable = true)



In [19]:
#solution2

from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType,FloatType

def dolariser(x):
  return str(x[1:-1])


udf_dolariser=udf(lambda x:dolariser(x),StringType())
price_df=chio.withColumn("item_price",udf_dolariser(col("item_price")))
price_df.printSchema()
price_df=price_df.withColumn("item_price",price_df["item_price"].cast("float"))
price_df.show()
price_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|      2.39|
|       1|       1|                Izze|        [Clementine]|      3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|      3.39|
|       1|       1|Chips and Tomatil...|                NULL|      2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|     10.98|
|       3|       1|       Side of Chips|                NULL|      1.69|
|       4|       1|       Steak Burrito|[Tomatill

In [20]:
#Step 14. How much was the revenue for the period in the dataset?

revenue_result=price_df.withColumn("revenue",price_df["item_price"] * price_df["quantity"])
total_revenue=revenue_result.agg({"revenue":"sum"})
total_revenue.show()

+----------------+
|    sum(revenue)|
+----------------+
|39237.0197327137|
+----------------+



In [21]:
print("Total Revenue: $",total_revenue.head()[0])

Total Revenue: $ 39237.0197327137


In [22]:
#Step 15. How many orders were made in the period?
from pyspark.sql.functions import countDistinct

total_orders = price_df.select(countDistinct("order_id"))
total_orders.show()
total_orders.head()
total_orders.head(1)[0]

print("Total Orders: ",total_orders.head(1)[0][0])

+------------------------+
|count(DISTINCT order_id)|
+------------------------+
|                    1834|
+------------------------+

Total Orders:  1834


In [23]:
#Step 16. What is the average revenue amount per order?
avg_revenue = revenue_result.groupBy("order_id").sum()

In [24]:
avg_revenue.show(5)

+--------+-------------+-------------+------------------+------------------+
|order_id|sum(order_id)|sum(quantity)|   sum(item_price)|      sum(revenue)|
+--------+-------------+-------------+------------------+------------------+
|     148|          592|            4|  30.9399995803833|  30.9399995803833|
|     463|          926|            2|10.679999828338623|10.679999828338623|
|     471|         2355|            5|24.149999618530273|24.149999618530273|
|     496|         2480|            5|17.549999952316284|17.549999952316284|
|     833|         1666|            2|             12.75|             12.75|
+--------+-------------+-------------+------------------+------------------+
only showing top 5 rows



In [25]:
avg_revenue

DataFrame[order_id: int, sum(order_id): bigint, sum(quantity): bigint, sum(item_price): double, sum(revenue): double]

In [26]:
avg_revenue.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+
|summary|         order_id|     sum(order_id)|     sum(quantity)|   sum(item_price)|      sum(revenue)|
+-------+-----------------+------------------+------------------+------------------+------------------+
|  count|             1834|              1834|              1834|              1834|              1834|
|   mean|            917.5|2336.8440567066523| 2.711014176663032| 18.81142843757617|21.394231042919138|
| stddev|529.5745147443052|1861.6693582520409|1.6776242381295468|11.652511679485094|30.216891178957045|
|    min|                1|                 2|                 1|10.079999804496765|10.079999804496765|
|    max|             1834|             21298|                35|            205.25|1074.2399997711182|
+-------+-----------------+------------------+------------------+------------------+------------------+



In [27]:
avg_revenue.describe().select('sum(revenue)').head(2)[1][0]

'21.394231042919138'

In [28]:
#Step 17.How many different items are sold?
total_items = price_df.select(countDistinct("item_name"))
total_items.show()

+-------------------------+
|count(DISTINCT item_name)|
+-------------------------+
|                       50|
+-------------------------+

